In [1]:
# !pip install tensorflow
# !pip install Keras-Preprocessing

In [2]:
import os
import pandas as pd
import numpy as np

from pyvi.ViTokenizer import ViTokenizer #vietnamese tokenizer

import re #regular expression

In [3]:
import pickle
from keras_preprocessing.sequence import pad_sequences

# from tensorflow.keras.models import Sequential 

from tensorflow.keras import layers, Sequential

In [4]:
import tensorflow as tf
# from keras.preprocessing import text, sequence, tokenizer

from tensorflow.keras.preprocessing import text, sequence 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Conv2D, MaxPool2D, Bidirectional, LSTM, GRU, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D, Reshape, Dropout, Concatenate
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

from keras.layers import Input, Dense, Embedding, Flatten, Conv2D, MaxPool2D, Bidirectional, LSTM, GRU, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D, Reshape, Flatten, Dropout, Concatenate

In [5]:
DIR_DATASET = os.path.join('dataset')

PATH_TRAIN = os.path.join(DIR_DATASET, 'core-data\\train.csv')
PATH_DEV = os.path.join(DIR_DATASET, 'core-data\dev.csv')
PATH_TEST = os.path.join(DIR_DATASET, 'core-data\\test.csv')

EMBEDDING_PATH = os.path.join(DIR_DATASET, 'cc.vi.300.vec')
TOKENIZER_PATH = os.path.join(DIR_DATASET, 'tokenizer.pickle')
VISTOPWORDS_PATH = os.path.join(DIR_DATASET, "vietnamese-stopwords.txt")

MODEL_DIR = 'model'
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [6]:
data_train = pd.read_csv(PATH_TRAIN)

data_dev = pd.read_csv(PATH_DEV)

data_test = pd.read_csv(PATH_TEST)

In [7]:
with open(VISTOPWORDS_PATH, "r", encoding="utf-8") as ins:
    stopwords = []
    for line in ins:
        dd = line.strip('\n')
        stopwords.append(dd)
    stopwords = set(stopwords)

In [8]:
# read csv data
data_train = pd.read_csv(PATH_TRAIN)
data_dev = pd.read_csv(PATH_DEV)
data_test = pd.read_csv(PATH_TEST)

In [9]:
data_train.columns

Index(['link', 'rating', 'comment', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'label', 'spam_label'],
      dtype='object')

In [10]:
data_train=data_train.drop(['link', 'rating', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'spam_label'], axis=1)
data_dev=data_dev.drop(['link', 'rating', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'spam_label'], axis=1)
data_test=data_test.drop(['link', 'rating', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'spam_label'], axis=1)

In [11]:
data_train.head(10)

,comment,label
0,sản phẩm đúng quảng cáo. giao hàng hơi chậm nh...,0
1,"Mình nhận hàng ngày hôm qua, tiki giao hàng đú...",0
2,"Máy giặt quá tốt, chỉ có 5 tr được cái máy giặ...",0
3,chất liệu vải tốt mát nói chung là chất lượng ...,0
4,Tủ tốt giá tốt nhiều hơn so với các trung tâm...,0
5,Mọi người dưới 56kg nên mua sz S thoii nha. Fo...,1
6,giao nhanh đóng hàng cẩn thận ăn ngon lắm,0
7,10 điểm. Giao hàng nhanh. Cầu hình cao màn hìn...,0
8,Bé nhà mình tiêu thụ loại này tốt!! Tuy nhiên ...,0
9,Ỏ Bến Tre mà vừa đặt hàng hôm qua nay nhận dc ...,0


In [12]:
dup_train = data_train.duplicated().sum()
dup_dev = data_dev.duplicated().sum()
dup_test = data_test.duplicated().sum()

print("train's duplicates: ", dup_train,
    "\ndev's duplicates: ", dup_dev, "\ntest's duplicates: ", dup_test)

train's duplicates:  8 
dev's duplicates:  0 
test's duplicates:  3


In [13]:
data_train=data_train.drop_duplicates(keep='first')
data_dev=data_dev.drop_duplicates(keep='first')
data_test=data_test.drop_duplicates(keep='first')

In [14]:
dup_train = data_train.duplicated().sum()
dup_dev = data_dev.duplicated().sum()
dup_test = data_test.duplicated().sum()

print("train's duplicates: ", dup_train,
    "\ndev's duplicates: ", dup_dev, "\ntest's duplicates: ", dup_test)

train's duplicates:  0 
dev's duplicates:  0 
test's duplicates:  0


PREPROCESS FUNCTION

In [15]:
# filder stop word
def filter_stop_words(train_sentences, stop_words):
    new_sent = [word for word in train_sentences.split() if word not in stop_words]
    train_sentences = ' '.join(new_sent)
    return train_sentences

# remove emoji
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
# declare func and adding some remove
def preprocess(text, tokenized = True, lowercased = True):
    text = ViTokenizer.tokenize(text) if tokenized else text
    text = filter_stop_words(text, stopwords)
    text = deEmojify(text)
    text = text.lower() if lowercased else text

    text = text.strip()
    text = re.compile('<.*?>').sub('', text)
    text = re.sub('\s+', ' ', text) 
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text

def pre_process_features(X, y, tokenized = True, lowercased = True):
    X = np.array(X)
    y = np.array(y)
    X = [preprocess(str(p), tokenized = tokenized, lowercased = lowercased) for p in list(X)]
    for idx, ele in enumerate(X):
        if not ele:
            X = np.delete(X, idx)
            y = np.delete(y, idx)
    return X, y


In [16]:
text_01 = "[HN] Hiện tại bên mình vẫn đang cần tuyển vị trí C# developer (.NET framework, C# biết winforms (2-3 năm)) sẵn sàng làm thêm FE hoặc Python, Java nếu được giao. Lương thỏa thuận, có 13-15 tháng lương. Quan tâm ib lấy JD."
text_02 = preprocess(text_01)
print("After preprocess: ", text_01, "\n")
print("Before preprocess: ", text_02)

After preprocess:  [HN] Hiện tại bên mình vẫn đang cần tuyển vị trí C# developer (.NET framework, C# biết winforms (2-3 năm)) sẵn sàng làm thêm FE hoặc Python, Java nếu được giao. Lương thỏa thuận, có 13-15 tháng lương. Quan tâm ib lấy JD. 

Before preprocess:   hn hiện_tại tuyển vị_trí c developer net framework c winforms sẵn_sàng fe python java giao lương thỏa_thuận lương quan_tâm ib jd


PREPROCESS

In [17]:
data_train

,comment,label
0,sản phẩm đúng quảng cáo. giao hàng hơi chậm nh...,0
1,"Mình nhận hàng ngày hôm qua, tiki giao hàng đú...",0
2,"Máy giặt quá tốt, chỉ có 5 tr được cái máy giặ...",0
3,chất liệu vải tốt mát nói chung là chất lượng ...,0
4,Tủ tốt giá tốt nhiều hơn so với các trung tâm...,0
...,...,...
14301,dùng okee ko gây nóng máy nhưng chất của ốp om...,0
14302,"Rất tốt, rất đáng tiền, nằm rất êm, lúc mới mu...",0
14303,"giao đủ, đóng gói chắc chắn, giao hàng nhanh \...",1
14304,giao hỏa tốc rẻ và nhanh. Nhưng trước đó mình ...,0


In [18]:
# slit data so that X_train is review and y_train is label
X_train = data_train.iloc[:, 0:1]
y_train = data_train.iloc[:, 1:2]

X_dev = data_dev.iloc[:, 0:1]
y_dev = data_dev.iloc[:, 1:2]

X_test = data_test.iloc[:, 0:1]
y_test = data_test.iloc[:, 1:2]    

In [19]:
y_train

,label
0,0
1,0
2,0
3,0
4,0
...,...
14301,0
14302,0
14303,1
14304,0


In [20]:
X_dev

,comment
0,"Dùng tốt, cắt lông mũi phù hợp"
1,5 câu kinh điển mà một người vợ thông minh sẽ ...
2,Hộp hơi móp chắc do vận chuyển ... Giao hàng l...
3,"Quần rất ổn, vải mát, mềm, giá rẻ, quá ưng.\r\..."
4,"Nhìn khá ổn, chất lượng thì chưa nói trước điề..."
...,...
1585,"Tốt, dây chắc chắn, đầu mạng tốt"
1586,"hàng tốt, *** thử các loại nước khác nhau thì ..."
1587,great quality
1588,"Lấy 2 bộ 1 đen 1 trắng, bộ đen rất hài lòng, b..."


In [21]:
# transform X_train into narray (train_X) and simular
train_X, train_y= pre_process_features(X_train['comment'], y_train['label'], tokenized=True, lowercased = True)
dev_X, dev_y= pre_process_features(X_dev['comment'], y_dev['label'], tokenized=True, lowercased = True)
test_X, test_y = pre_process_features(X_test['comment'], y_test['label'], tokenized=True, lowercased = True)

In [22]:
np.save('dataset/internal-vocabulary.npy', train_X)

In [23]:
train_X.shape

(14271,)

In [24]:
train_y.shape

(14271,)

DECLARE FUNCTIONS

In [25]:
embeddings_index = {}
with open(EMBEDDING_PATH, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [26]:
tokenizer = text.Tokenizer(lower=False, filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~\t\n')
tokenizer.fit_on_texts(train_X)

In [27]:
with open(TOKENIZER_PATH, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
# def make_featues(X, y, tokenizer, is_one_hot_label=True, number_class1=2):
#     X = tokenizer.texts_to_sequences(X)
#     X = pad_sequences(X, maxlen=sequence_length)
#     if is_one_hot_label: 
#         y = to_categorical(y, num_classes=number_class1)
#     return X, y

def make_featues(X, y, tokenizer, is_one_hot_label=False, number_class1=2): # my model just binary (non-spam or not spam)
    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=sequence_length)
    if is_one_hot_label: 
        y = to_categorical(y, num_classes=number_class1)
    return X, y

In [29]:
vocabulary_size = 10000
sequence_length = 100

embedding_dim = 300
batch_size = 256
epochs = 40
drop = 0.5

filter_sizes = [2,3,5]
num_filters = 32

word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i >= vocabulary_size:
        continue

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [30]:
train_XX, train_yy = make_featues(train_X, train_y, tokenizer)
dev_XX, dev_yy = make_featues(dev_X, dev_y, tokenizer)
test_XX, test_yy = make_featues(test_X, test_y, tokenizer)

In [31]:
train_yy

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [32]:
train_XX.shape

(14271, 100)

In [33]:
print(train_XX[5, :])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0  268  141    3  710 6622   18
  347   49  414  497   13   65   82 6623    4    7  711   37  119 2741
  498  831]


In [34]:
print(type(train_XX), train_XX.shape)
print(type(train_yy), train_yy.shape)
print(type(dev_XX), dev_XX.shape)
print(type(dev_yy), dev_yy.shape)
print(type(test_XX), test_XX.shape)
print(type(test_yy), test_yy.shape)

<class 'numpy.ndarray'> (14271, 100)
<class 'numpy.ndarray'> (14271,)
<class 'numpy.ndarray'> (1589, 100)
<class 'numpy.ndarray'> (1589,)
<class 'numpy.ndarray'> (3962, 100)
<class 'numpy.ndarray'> (3962,)


In [35]:
train_XX.shape

(14271, 100)

CNNs MODEL

In [36]:
# TextCNN
# init layers
textCNNModel = Sequential()

# emmbed word vector
textCNNModel.add(layers.Embedding(num_words, embedding_dim))

# model
textCNNModel.add(layers.Conv1D(128, 5, activation='relu'))
textCNNModel.add(layers.GlobalMaxPooling1D())
textCNNModel.add(layers.Dense(256, activation='relu'))

# Output layer for binary classification
textCNNModel.add(layers.Dense(1, activation='sigmoid')) 

# Compile the model
textCNNModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

textCNNModel.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ ?                           │     0 (unbuilt) │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [37]:
# Test for model accuracy
textCNNModel.fit( train_XX, train_yy,
    epochs=epochs,
    verbose=2,
    validation_data=(dev_XX, dev_yy),
    batch_size=batch_size)

Epoch 1/40
56/56 - 11s - 201ms/step - accuracy: 0.7732 - loss: 0.4978 - val_accuracy: 0.8414 - val_loss: 0.3941
Epoch 2/40
56/56 - 10s - 174ms/step - accuracy: 0.8733 - loss: 0.3144 - val_accuracy: 0.8383 - val_loss: 0.3771
Epoch 3/40
56/56 - 10s - 175ms/step - accuracy: 0.9263 - loss: 0.1926 - val_accuracy: 0.8062 - val_loss: 0.4550
Epoch 4/40
56/56 - 10s - 176ms/step - accuracy: 0.9649 - loss: 0.1049 - val_accuracy: 0.8433 - val_loss: 0.5607
Epoch 5/40
56/56 - 9s - 162ms/step - accuracy: 0.9815 - loss: 0.0548 - val_accuracy: 0.8320 - val_loss: 0.6120
Epoch 6/40
56/56 - 9s - 154ms/step - accuracy: 0.9879 - loss: 0.0365 - val_accuracy: 0.8357 - val_loss: 0.7431
Epoch 7/40
56/56 - 9s - 154ms/step - accuracy: 0.9910 - loss: 0.0256 - val_accuracy: 0.8376 - val_loss: 0.7425
Epoch 8/40
56/56 - 9s - 152ms/step - accuracy: 0.9938 - loss: 0.0181 - val_accuracy: 0.8402 - val_loss: 0.7888
Epoch 9/40
56/56 - 9s - 153ms/step - accuracy: 0.9946 - loss: 0.0162 - val_accuracy: 0.8326 - val_loss: 0.90

In [38]:
# Evaluate the model on the TEST data
loss, accuracy = textCNNModel.evaluate(test_XX, test_yy)
print(f"LOSS: {loss}")
print ('Accuracy '+ str(accuracy))

124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8230 - loss: 1.2622
LOSS: 1.2615290880203247
Accuracy 0.818778395652771


LSTM MODEL

In [39]:
# LSTM
# init layer
lstmModel = Sequential()

# emmbed word vectors
lstmModel.add( layers.Embedding( num_words, embedding_dim) )
lstmModel.add( layers.SpatialDropout1D(0.2) )

# lstmModel.add( layers.LSTM( 100, return_sequences=False, dropout=0.5, recurrent_dropout=0.5 ) )
lstmModel.add( layers.LSTM( 100, return_sequences=False ) )
lstmModel.add( layers.Dense( 512, activation="relu" ) )

# Output layer for binary classification
lstmModel.add( layers.Dense( 1, activation="sigmoid" ) )

# Compile the model
lstmModel.compile( optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

lstmModel.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [40]:
# utils.plot_model(lstmModel)
lstmModel.fit( train_XX, train_yy,
    epochs=epochs,
    verbose=2,
    validation_data=(dev_XX, dev_yy),
    batch_size=batch_size)

Epoch 1/40
56/56 - 16s - 291ms/step - accuracy: 0.7747 - loss: 0.4962 - val_accuracy: 0.8326 - val_loss: 0.4142
Epoch 2/40
56/56 - 14s - 255ms/step - accuracy: 0.8619 - loss: 0.3291 - val_accuracy: 0.8420 - val_loss: 0.4044
Epoch 3/40
56/56 - 13s - 237ms/step - accuracy: 0.9024 - loss: 0.2459 - val_accuracy: 0.8357 - val_loss: 0.4398
Epoch 4/40
56/56 - 13s - 233ms/step - accuracy: 0.9280 - loss: 0.1832 - val_accuracy: 0.8307 - val_loss: 0.4756
Epoch 5/40
56/56 - 13s - 235ms/step - accuracy: 0.9474 - loss: 0.1410 - val_accuracy: 0.8206 - val_loss: 0.5278
Epoch 6/40
56/56 - 13s - 232ms/step - accuracy: 0.9558 - loss: 0.1164 - val_accuracy: 0.8263 - val_loss: 0.6002
Epoch 7/40
56/56 - 13s - 239ms/step - accuracy: 0.9634 - loss: 0.0979 - val_accuracy: 0.8244 - val_loss: 0.6350
Epoch 8/40
56/56 - 13s - 237ms/step - accuracy: 0.9660 - loss: 0.0886 - val_accuracy: 0.8206 - val_loss: 0.7044
Epoch 9/40
56/56 - 13s - 235ms/step - accuracy: 0.9692 - loss: 0.0775 - val_accuracy: 0.8188 - val_loss:

In [41]:
# Evaluate the model on the DEV data
loss, accuracy = lstmModel.evaluate(test_XX, test_yy)
print(f"LOSS: {loss}")
print ('Accuracy '+ str(accuracy))

124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8075 - loss: 1.7562
LOSS: 1.8377150297164917
Accuracy 0.8028773069381714


SAVE MODEL

In [42]:
CNNs = os.path.join(MODEL_DIR, 'CNNs.keras')  
LSTM = os.path.join(MODEL_DIR, 'LSTM.keras')

textCNNModel.save(CNNs)
lstmModel.save(LSTM)